In [8]:
import json
import pandas as pd
from datetime import datetime




In [9]:
# Load the JSON data
with open('./Fit/All data/raw_com.google.step_count.delta_com.google.and.json', 'r') as file:
    data = json.load(file)

# Extract the data points
data_points = data['Data Points']



In [10]:
# Create a list to store the transformed data
transformed_data = []

# Process each data point
for point in data_points:
    steps = point['fitValue'][0]['value']['intVal']
    start_time = int(point['startTimeNanos'])
    end_time = int(point['endTimeNanos'])
    
    # Convert nanoseconds to datetime
    start_datetime = datetime.fromtimestamp(start_time / 1e9)
    end_datetime = datetime.fromtimestamp(end_time / 1e9)
    
    # Append the data to the list
    transformed_data.append({
        'steps': steps,
        'start_time': start_datetime,
        'end_time': end_datetime
    })

# Create a DataFrame from the list
df = pd.DataFrame(transformed_data)


In [11]:
# Aggregate by day
df['date'] = df['start_time'].dt.date
daily_steps = df.groupby('date')['steps'].sum().reset_index()

print(daily_steps)


          date   steps
0   2024-06-08    3615
1   2024-06-09    2491
2   2024-06-10   18051
3   2024-06-11   13049
4   2024-06-12   16118
5   2024-06-13   13301
6   2024-06-14   16142
7   2024-06-15    1950
8   2024-06-16    8883
9   2024-06-17   18681
10  2024-06-18    9249
11  2024-06-19    9895
12  2024-06-20   11682
13  2024-06-21    2262
14  2024-06-22    2939
15  2024-06-23    9574
16  2024-06-24   16989
17  2024-06-25    9963
18  2024-06-26    9079
19  2024-06-27   16190
20  2024-06-28   13895
21  2024-06-29    2641
22  2024-06-30    9679
23  2024-07-01    8469
24  2024-07-02    8051
25  2024-07-03    8659
26  2024-07-04   12646
27  2024-07-05    7947
28  2024-07-06    4552
29  2024-07-07    2105
30  2024-07-08    9542
31  2024-07-09   15188
32  2024-07-10    8000
33  2024-07-11  196007


In [12]:
import os
import pandas as pd
from fit_data_extraction import extract_fit_data  # Import the function from your saved file

# Function to integrate Fit data into existing DataFrame
def integrate_fit_data(fit_dir):
    data_points = []  # List to store data points

    # Iterate through each file in the directory
    for filename in os.listdir(fit_dir):
        if filename.endswith('.json'):
            file_path = os.path.join(fit_dir, filename)
            data_point = extract_fit_data(file_path)

            # Assuming you also want to capture the date from filename or another source
            # Update this part based on your actual data extraction logic
            date = filename.split('.')[0]  # Example: Extract date from filename

            data_point['Date'] = date
            data_points.append(data_point)

    # Create DataFrame from list of dictionaries
    fit_data = pd.DataFrame(data_points, columns=['Date', 'Fitness Activity', 'Start Time', 'End Time', 'Duration'])
    return fit_data

# Directory containing Fit JSON files
fit_dir = './Fit/All sessions/'

# Integrate Fit data into DataFrame
fit_data = integrate_fit_data(fit_dir)

# Display the extracted data
print(fit_data.head())


                  Date Fitness Activity                Start Time  \
0  2024-05-01T12_17_18     calisthenics  2024-05-01T00:17:18.408Z   
1  2024-05-09T12_19_08     calisthenics  2024-05-09T00:19:08.139Z   
2  2024-06-27T15_10_16          walking  2024-06-27T03:10:16.123Z   
3  2024-05-29T07_45_50     calisthenics  2024-05-28T19:45:50.496Z   
4  2024-02-24T12_29_21     calisthenics  2024-02-23T23:29:21.233Z   

                   End Time   Duration  
0  2024-05-01T00:27:53.242Z   634.834s  
1  2024-05-09T00:36:23.880Z  1035.741s  
2  2024-06-27T03:40:16.123Z      1800s  
3  2024-05-28T19:46:13.262Z    22.766s  
4  2024-02-23T23:46:47.822Z  1046.589s  
